# Assemble Garcia & Keeling 2001 O$_2$ flux product

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
from datetime import datetime

import numpy as np
import xarray as xr

from netCDF4 import default_fillvals

import matplotlib.pyplot as plt

import config
import fgo2_gk2001

### Generate datasets

### Shifting and scaling
The recommended Garcia and Keeling rescaling is 0.82 
(Naegler 2006 vs. Wanninkhof 1992 scaling factors 0.32/0.39), plus a 10-day shift later, to allow for mixed layer equilibration.

In [ ]:
ds = fgo2_gk2001.open_flux_dataset(shift_time=10., scaleby=0.82, clobber=True)
ds

In [ ]:
ds.fgo2_sea.mean("lon").T.plot()

In [ ]:
dsets_ann = {}
for product in ["Gruber", "Resplandy"]:
    fig = plt.figure()    
    ds_ann = fgo2_gk2001.open_inversion_annual_flux(product, gk_grid=True)
    ds_ann.fgo2.plot(vmin=-8., vmax=8., cmap='RdBu_r')
    dsets_ann[product] = ds_ann
    glb_sum = (ds_ann.fgo2 * ds_ann.area).sum() / 1e12
    plt.title(f"{glb_sum.values:0.4f} Tmol/y")
    fig.suptitle(product, fontweight="bold")    
ds_ann.fgo2  

### Make some quick plots of the fluxes

## Apply annual mean fluxes from Gruber et al. 2001 & Resplandy et al. 2016

In [ ]:
dirout = config.flux_product_dir
clobber = True
dsets = {}
for product in ["Gruber", "Resplandy"]:

    dso = ds[["fgo2_sea", "fgo2_thm_sea", "fgo2_bio_sea", "area"]]    
    dso["fgo2"] = xr.full_like(ds.fgo2_sea, fill_value=0.)
    dso.fgo2.values = ds.fgo2_sea + dsets_ann[product].fgo2
    dso.fgo2.attrs["long_name"] = f"GK01 O$_2$ seasonal flux w/ {product} ann mean"
    dso.fgo2.encoding["_FillValue"] =  default_fillvals["f8"]

    for v in dso.data_vars:
        if "_FillValue" not in dso[v].encoding:
            dso[v].encoding["_FillValue"] = None

    dsets[product] = dso
    
    file_out = f"{dirout}/garcia-keeling.fgo2.{product.lower()}_ann.nc"    
    if clobber and os.path.exists(file_out):
        os.remove(file_out)
    if not os.path.exists(file_out):
        dso.to_netcdf(file_out)